# 产品消息图形展示

## 获取数据

设置使用的数据源`engine`，`nmc_monitor`表示使用从NMC监控平台中获取的消息数据。关于消息数据的格式及消息的生成和存储工具请访问[nwpc-oper/nmc-message-client](https://github.com/nwpc-oper/nmc-message-client)。

构建起报时次列表`start_time`，从2020年3月11日12时次到3月21日12时次。

设置系统名称`system`，`nwpc_grapes_gfs`代表GRAPES GFS全球预报系统。

设置ElasticSearch服务器网址`elastic_server`。

In [ ]:
import pandas as pd
from nwpc_message_tool import nmc_monitor

engine = nmc_monitor


time_series = pd.date_range(
    start="2020-03-11T12:00:00",
    end="2020-03-21T23:00:00",
    freq="D",
)

start_time = list(s.to_pydatetime() for s in time_series)

system = "nwpc_grapes_gfs"
system = engine.fix_system_name(system)

elastic_server = "localhost:9200"

从ElasticSearch服务器中获取消息。

In [8]:
from nwpc_message_tool.storage import EsMessageStorage

client = EsMessageStorage(
    hosts=elastic_server,
    engine=engine,
)

results = client.get_production_messages(
    system=system,
    production_stream="oper",
    production_type="grib2",
    production_name="orig",
    start_time=start_time
)

results = list(results)

2020-03-22 20:33:11.466 | INFO     | nwpc_message_tool.storage:get_production_messages:70 - found results: 671
100%|██████████| 671/671 [00:00<00:00, 4066.68it/s]


使用表格处理器`TableProcessor`将原始数据转换为`pandas.DataFrame`格式的表格数据。

In [10]:
from nwpc_message_tool.processor import TableProcessor

processor = TableProcessor()
df = processor.process_messages(results)
df

2020-03-22 20:33:22.689 | INFO     | nwpc_message_tool.processor.table_processor:process_messages:55 - searching...done
2020-03-22 20:33:22.690 | INFO     | nwpc_message_tool.processor.table_processor:process_messages:57 - get 671 results


,system,stream,type,name,start_time,forecast_hour,time,event,status
2020031112+000,nwpc_grapes_gfs,oper,grib2,orig,2020-03-11 12:00:00+00:00,0,2020-03-11 16:16:12+00:00,before_upload,Complete
2020031112+003,nwpc_grapes_gfs,oper,grib2,orig,2020-03-11 12:00:00+00:00,3,2020-03-11 16:16:59+00:00,before_upload,Complete
2020031112+006,nwpc_grapes_gfs,oper,grib2,orig,2020-03-11 12:00:00+00:00,6,2020-03-11 16:17:23+00:00,before_upload,Complete
2020031112+009,nwpc_grapes_gfs,oper,grib2,orig,2020-03-11 12:00:00+00:00,9,2020-03-11 16:17:43+00:00,before_upload,Complete
2020031112+012,nwpc_grapes_gfs,oper,grib2,orig,2020-03-11 12:00:00+00:00,12,2020-03-11 16:18:06+00:00,before_upload,Complete
...,...,...,...,...,...,...,...,...,...
2020032112+216,nwpc_grapes_gfs,oper,grib2,orig,2020-03-21 12:00:00+00:00,216,2020-03-21 16:40:24+00:00,before_upload,Complete
2020032112+222,nwpc_grapes_gfs,oper,grib2,orig,2020-03-21 12:00:00+00:00,222,2020-03-21 16:41:54+00:00,before_upload,Complete
2020032112+228,nwpc_grapes_gfs,oper,grib2,orig,2020-03-21 12:00:00+00:00,228,2020-03-21 16:41:54+00:00,before_upload,Complete
2020032112+234,nwpc_grapes_gfs,oper,grib2,orig,2020-03-21 12:00:00+00:00,234,2020-03-21 16:42:23+00:00,before_upload,Complete


下面介绍如何使用该表格数据绘制图形。

## 逐时效产品生成时间网格图

使用`StepGridPlotPresenter`绘制时效网格图。每个网格代表一个时效的GRIB2产品生成时间，颜色越深代表生成时间越晚。

**注意**：`Presenter`对象会修改原始数据，如果后续需要继续使用原始表格数据，请在调用`show()`函数时传入数据的拷贝。

In [12]:
from nwpc_message_tool.presenter import StepGridPlotPresenter

presenter = StepGridPlotPresenter(
    system=system,
    output_type=("notebook",),
)
presenter.show(df.copy())

Loading BokehJS ...

## 产品生成时间段条形图

使用`CyclePeriodPlotPresenter`生成时间段条形图。左端表示第一个GRIB2产品生成时间，右端表示最后一个GRIB2产品生成时间。

In [13]:
from nwpc_message_tool.presenter import CyclePeriodPlotPresenter

presenter = CyclePeriodPlotPresenter(
    system=system,
    output_type=("notebook",),
)
presenter.show(df.copy())

Loading BokehJS ...